In [1]:
from config import fund_dict
import requests
import pandas as pd
from helper import create_holdings_df, process_scraped_data
import traceback
import numpy as np
import datetime


In [2]:
max_num_of_rows = 35
df_all = pd.DataFrame()
for fund in fund_dict:
    try:
        fund_name = fund
        cik = fund_dict[fund].split('CIK=')[1].split('&')[0]
        df_fund = create_holdings_df(cik_=cik,fund_name_=fund_name)
        df_all = pd.concat([df_all, df_fund]).reset_index(drop=True)
        print('Fund: {} CIK: {} added to df  '.format(fund_name, cik))
        #print(df_all.tail())
    except:
        print('Issue with: ', fund)
        print(traceback.format_exc())
        continue

https://data.sec.gov/submissions/CIK0001358706.json
https://www.sec.gov/Archives/edgar/data/1358706/0001172661-23-002975.txt
https://www.sec.gov/Archives/edgar/data/1358706/0001172661-23-002081.txt
Fund: ABRAMS CAPITAL MANAGEMENT, L.P. CIK: 1358706 added to df  
https://data.sec.gov/submissions/CIK0001112520.json
https://www.sec.gov/Archives/edgar/data/1112520/0001112520-23-000013.txt
https://www.sec.gov/Archives/edgar/data/1112520/0001112520-23-000008.txt
Fund: AKRE CAPITAL MANAGEMENT CIK: 1112520 added to df  
https://data.sec.gov/submissions/CIK0001491072.json
https://www.sec.gov/Archives/edgar/data/1491072/0000950123-23-008259.txt
https://www.sec.gov/Archives/edgar/data/1491072/0000950123-23-005475.txt
Fund: ANSON FUNDS CIK: 1491072 added to df  
https://data.sec.gov/submissions/CIK0001656456.json
https://www.sec.gov/Archives/edgar/data/1656456/0001656456-23-000003.txt
https://www.sec.gov/Archives/edgar/data/1656456/0001656456-23-000002.txt
Fund: APPALOOSA LP CIK: 1656456 added to 

In [15]:
df_all.to_csv('data_by_stocks.csv',index=False)
most_recent_filing_date = str(df_all.filingDate.max())
print('most recent filing date: ', most_recent_filing_date)

print(df_all.columns)
df_all.tail()

most recent filing date:  2023-08-14
Index(['nameOfIssuer', 'titleOfClass', 'cusip', 'value', 'shrsOrPrnAmt',
       'investmentDiscretion', 'votingAuthority', 'sshPrnamt', 'sshPrnamtType',
       'cik', 'filingDate', 'reportDate', 'fund_name', 'figi', 'putCall',
       'otherManager', 'nameOfIssuer_link'],
      dtype='object')


,nameOfIssuer,titleOfClass,cusip,value,shrsOrPrnAmt,investmentDiscretion,votingAuthority,sshPrnamt,sshPrnamtType,cik,filingDate,reportDate,fund_name,figi,putCall,otherManager,nameOfIssuer_link
9132,TWELVE SEAS INVESTMENT CO II,*W EXP 03/02/202,90118T114,9485,"{'sshPrnamt': '190848', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '190848', 'Shared': '0', 'None': '0'}",190848,SH,1348145,2023-05-15,2023-03-31,VR ADVISORY SERVICES,NaN,NaN,NaN,"<a href=""http://www.google.com/search?q=stock ..."
9133,VELO3D INC,*W EXP 09/29/202,92259N112,38610,"{'sshPrnamt': '100000', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '100000', 'Shared': '0', 'None': '0'}",100000,SH,1348145,2023-05-15,2023-03-31,VR ADVISORY SERVICES,NaN,NaN,NaN,"<a href=""http://www.google.com/search?q=stock ..."
9134,VIEW INC,*W EXP 03/08/202,92671V114,4074,"{'sshPrnamt': '105000', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '105000', 'Shared': '0', 'None': '0'}",105000,SH,1348145,2023-05-15,2023-03-31,VR ADVISORY SERVICES,NaN,NaN,NaN,"<a href=""http://www.google.com/search?q=stock ..."
9135,YANDEX N V,SHS CLASS A,N97284108,523558,"{'sshPrnamt': '429023', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '429023', 'Shared': '0', 'None': '0'}",429023,SH,1348145,2023-05-15,2023-03-31,VR ADVISORY SERVICES,NaN,NaN,NaN,"<a href=""http://www.google.com/search?q=stock ..."
9136,YPF SOCIEDAD ANONIMA,SPON ADR CL D,984245100,9902730,"{'sshPrnamt': '901888', 'sshPrnamtType': 'SH'}",SOLE,"{'Sole': '901888', 'Shared': '0', 'None': '0'}",901888,SH,1348145,2023-05-15,2023-03-31,VR ADVISORY SERVICES,NaN,NaN,NaN,"<a href=""http://www.google.com/search?q=stock ..."


In [30]:
import requests
import pandas as pd
import xmltodict
from datetime import datetime, timedelta

def process_scraped_data(df_all_):
    lookback_period = 90
    df_all_['nameOfIssuer_link'] =df_all_['nameOfIssuer'].apply(
    lambda x: str('''<a href="http://www.google.com/search?q=stock price {}" 
        target="_blank">{}</a>'''.format(x, x)))
    reporting_periods = list(set(df_all_.reportDate.values))
    filing_periods = list(set(df_all_.filingDate.values))
    print('filing_periods:', filing_periods)
    print('reporting_periods:', reporting_periods)
    current_filign_periods = []
    previous_filing_periods = []

    ninty_days_ago = (datetime.today() - timedelta(days=lookback_period)).strftime('%Y-%m-%d')
    oneeighty_days_ago = (datetime.today() - timedelta(days=lookback_period*2)).strftime('%Y-%m-%d')
    print('ninty_days_ago: ',ninty_days_ago)
    for period in filing_periods:
        if period > ninty_days_ago:
            #periods.remove(period)
            current_filign_periods.append(period)
        elif ((period > oneeighty_days_ago) & (period < ninty_days_ago)):
            previous_filing_periods.append(period)
    filing_periods.sort(reverse=True)
    print('current_filign_periods: ', current_filign_periods)
    print('previous_filing_periods: ', previous_filing_periods)
    df_all_.reset_index().tail()
    #df_all_[df_all_.filingDate == filing_periods[0]].reset_index().tail()
    #df_current = df_all_[((df_all_.putCall.isnull()) & (df_all_.reportDate == periods[0]))] \
    df_current = df_all_[((df_all_.putCall.isnull()) & (df_all_.filingDate.isin(current_filign_periods)))] \
        .reset_index(drop=True)
    df_puts_current = df_all_[((df_all_.putCall=='Put') & (df_all_.filingDate.isin(current_filign_periods)))] \
        .reset_index(drop=True)
    #df_previous = df_all_[((df_all_.putCall.isnull()) & (df_all_.filingDate == periods[1]))] \
    df_previous = df_all_[((df_all_.putCall.isnull()) & (df_all_.filingDate.isin(previous_filing_periods)))] \
        .reset_index(drop=True)
    df_puts_current_g = df_puts_current.groupby(['nameOfIssuer','nameOfIssuer_link','cusip']) \
        .agg({'value':'sum','fund_name':lambda x: list(x)}).sort_values('value', ascending=False)
    df_current_g = df_current.groupby(['nameOfIssuer','nameOfIssuer_link','cusip']).sum().sort_values('value', ascending=False)
    df_previous_g = df_previous.groupby(['nameOfIssuer','nameOfIssuer_link','cusip']).sum().sort_values('value', ascending=False)
    df_diff = df_current_g - df_previous_g
    df_heavy_ = df_current_g
    df_hot_ = df_diff[df_diff.value > 0]
    df_cold_ = df_diff[df_diff.value < 0]
    
    return df_heavy_, df_hot_, df_cold_, df_puts_current_g
    

In [31]:
df_all.to_csv('data_by_stocks.csv',index=False)
most_recent_filing_date = str(df_all.filingDate.max())
print('most recent filing date: ', most_recent_filing_date)
most_recent_reporting_date = str(df_all.reportDate.max())
print('most recent reporting date: ', most_recent_reporting_date)

df_heavy, df_hot, df_cold, df_puts = process_scraped_data(df_all)

most recent filing date:  2023-08-14
most recent reporting date:  2023-06-30
filing_periods: ['2018-11-14', '2023-08-11', '2023-08-04', '2023-05-12', '2023-05-15', '2020-02-14', '2020-08-14', '2020-05-15', '2019-02-14', '2020-05-12', '2023-08-14']
reporting_periods: ['2023-03-31', '2018-09-30', '2019-12-31', '2020-06-30', '2018-12-31', '2023-06-30', '2020-03-31']
ninty_days_ago:  2023-07-30
current_filign_periods:  ['2023-08-11', '2023-08-04', '2023-08-14']
previous_filing_periods:  ['2023-05-12', '2023-05-15']


/var/folders/yb/_9tm4m2j26x6mhfjhf4r1v140000gn/T/ipykernel_12040/3453863059.py:42: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_current_g = df_current.groupby(['nameOfIssuer','nameOfIssuer_link','cusip']).sum().sort_values('value', ascending=False)
/var/folders/yb/_9tm4m2j26x6mhfjhf4r1v140000gn/T/ipykernel_12040/3453863059.py:43: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_previous_g = df_previous.groupby(['nameOfIssuer','nameOfIssuer_link','cusip']).sum().sort_values('value', ascending=False)


In [32]:
df_hot

value  \
nameOfIssuer                             nameOfIssuer_link                                  cusip                   
10X CAP VENTURE ACQSTN CORP              <a href="http://www.google.com/search?q=stock p... G87076108     21035.0   
10X CAPITAL VENTURE ACQ III              <a href="http://www.google.com/search?q=stock p... G87077106     72000.0   
                                                                                            G87077114    933799.0   
10X Capital Venture Acquisition Corp III <a href="http://www.google.com/search?q=stock p... G87077114      6495.0   
26 CAPITAL ACQUISITION CORP              <a href="http://www.google.com/search?q=stock p... 90138P118      5192.0   
...                                                                                                           ...   
ZOOM VIDEO COMMUNICATIONS IN             <a href="http://www.google.com/search?q=stock p... 98980L101  12768875.0   
ZSCALER INC                              <a href="http://www.google.com/search?q=stock p... 98980G102   4296385.0   
ZURA BIO LTD                             <a href="http://www.google.com/search?q=stock p... G9TY5A119      8629.0   
argenx SE                                <a href="http://www.google.com/search?q=stock p... 04016X101   4804727.0   
bluebird bio, Inc.                       <a href="http://www.google.com/search?q=stock p... 09609G100    274968.0   

                                                                                                       sshPrnamt  
nameOfIssuer                             nameOfIssuer_link                                  cusip                 
10X CAP VENTURE ACQSTN CORP              <a href="http://www.google.com/search?q=stock p... G87076108        0.0  
10X CAPITAL VENTURE ACQ III              <a href="http://www.google.com/search?q=stock p... G87077106        0.0  
                                                                                            G87077114    91102.0  
10X Capital Venture Acquisition Corp III <a href="http://www.google.com/search?q=stock p... G87077114        0.0  
26 CAPITAL ACQUISITION CORP              <a href="http://www.google.com/search?q=stock p... 90138P118        0.0  
...                                                                                                          ...  
ZOOM VIDEO COMMUNICATIONS IN             <a href="http://www.google.com/search?q=stock p... 98980L101   195767.0  
ZSCALER INC                              <a href="http://www.google.com/search?q=stock p... 98980G102    -6187.0  
ZURA BIO LTD                             <a href="http://www.google.com/search?q=stock p... G9TY5A119        0.0  
argenx SE                                <a href="http://www.google.com/search?q=stock p... 04016X101        0.0  
bluebird bio, Inc.                       <a href="http://www.google.com/search?q=stock p... 09609G100        0.0  

[1097 rows x 2 columns]